In [ ]:
import uuid
import random

from tqdm import tqdm
from threading import Thread
from threading import Thread, Lock

from cassandra.cluster import Cluster, BatchStatement
from cassandra.policies import RetryPolicy, ExponentialReconnectionPolicy

from utils.query_utils import *
from cassandra import ConsistencyLevel
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT

from cassandra.cluster import Cluster
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra.policies import WhiteListRoundRobinPolicy, DowngradingConsistencyRetryPolicy
from cassandra.query import tuple_factory
import time

In [ ]:
profile = ExecutionProfile(
    retry_policy = DowngradingConsistencyRetryPolicy,
    consistency_level = ConsistencyLevel.TWO,
    serial_consistency_level = ConsistencyLevel.LOCAL_SERIAL,
    request_timeout = 200,
)

CLUSTER_IDS = ['172.21.0.2']
KEYSPACE = 'library_keyspace'

cluster = Cluster(CLUSTER_IDS, execution_profiles={EXEC_PROFILE_DEFAULT: profile})
session = cluster.connect()
session.set_keyspace(KEYSPACE)

In [ ]:
res = get_all_reservations(session)
for r in res:
    reservation_id = r.reservation_id
    user_id = r.user_id
    user_name = r.user_name
    book_id = r.book_id
    book_name = r.book_name
    
reservation_id

In [ ]:
def add_batch_reservation(session, reservation_id, user_id, user_name, book_name, book_id, batch_size = 1000, forceful_exec = False, prepared = False):
    pass

def cancel_batch_reservation(session, reservation_id, batch_size = 1000, forceful_exec = False, prepared = False):
    pass

In [ ]:
def test_reserve_cancel(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)
        cancel_reservation(session, reservation_id)

    add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)

def test_reserve_cancel_fast(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):

    for _ in tqdm(range(repeat_num)):
        add_batch_reservation(session, reservation_id, user_id, user_name, book_name, book_id)
        cancel_batch_reservation(session, reservation_id)

    add_batch_reservation(session, reservation_id, user_id, user_name, book_name, book_id, forceful_exec = True)

In [ ]:
start_time = time.time()
test_reserve_cancel(session, reservation_id, user_id, user_name, book_id, book_name)
end_time = time.time()
print(f"Execution Time: {end_time - start_time}")

In [ ]:
start_time = time.time()
test_reserve_cancel_fast(session, reservation_id, user_id, user_name, book_id, book_name)
end_time = time.time()
print(f"Execution Time: {end_time - start_time}")